### ספריות


In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

### פונקציות גלובליות


In [3]:
from global_functions import remove_spaces_in_columns, up_load_df, first_15_division, equal_distribution_students, split_to_layers

### העלאת טבלאות


In [ ]:
# בתי ספר ממשרד החינוך בכל הארץ
Gschool=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'schools_2023')
Gschool = remove_spaces_in_columns(Gschool)
Gschool['מספר_תלמידים'] = Gschool['מספר_תלמידים'].fillna(0)

In [ ]:
# יישובים בשטח צתאל שם וקוד יישוב
JTMT_setls=up_load_df(r'{}\background_files'.format(education_folder_path),'210615_מקוצר_רשימת_יישובים_באחריות_צתאל')
JTMT_setls = remove_spaces_in_columns(JTMT_setls)

### עיבוד


In [6]:
# סינון ליישובים שנמצאים בשטח צתאל
Gschool=Gschool[Gschool['יישוב'].isin(JTMT_setls['שם_יישוב'])]

In [7]:
# הורדה של מוסדות בירושלים ובית שמש
Gschool=Gschool[Gschool['יישוב'] != 'ירושלים']
Gschool=Gschool[Gschool['יישוב'] != 'בית שמש']

In [ ]:
# קאורדינטות של מוסדות חינוך במרחב ירושלים
JTMT_setls_schools_coordinates_with_src=up_load_df(r'{}\background_files'.format(education_folder_path),'JTMT_setls_schools_coordinates_with_src')
JTMT_setls_schools_coordinates_with_src=remove_spaces_in_columns(JTMT_setls_schools_coordinates_with_src)

In [9]:
# בדיקה אם יש מוסדות עם אותו סמל מוסד
is_unique = JTMT_setls_schools_coordinates_with_src['סמל_מוסד'].is_unique

In [10]:
# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
Gschool = pd.merge(Gschool, JTMT_setls_schools_coordinates_with_src[['סמל_מוסד', 'coordinate_x', 'coordinate_y', 'SRC']],
                     on='סמל_מוסד', how='left')

# עדכון העמודות המקוריות
Gschool['coordinate_x'] = Gschool['coordinate_x'].fillna(0)
Gschool['coordinate_y'] = Gschool['coordinate_y'].fillna(0)

In [11]:
# מחיקת עמודות המיותרות
Gschool=Gschool.drop(columns=['כתובת_חט"ב', "טלפון", 'דוא"ל_מזכירות', "מעמד_משפטי"])

In [12]:
Gschool[['מכיתה', 'עד_כיתה']] = Gschool['שכבה'].str.split(' - ', expand=True)
Gschool.rename(columns={'מספר_תלמידים': 'סה"כ_תלמידים'}, inplace=True)

In [13]:
# הגדרת רשימת הכיתות לטווח גן עד יד'
class_map = ['גן' ,'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']

# הגדרת עמודות לכיתות עם NaN כערך ברירת מחדל
for single_class in class_map:
    Gschool[f'כיתה_{single_class}'] = np.nan

# פונקציה לחלוקה ראשונית של 15 תלמידים לכל כיתה בטווח
Gschool = Gschool.apply(lambda row: first_15_division(class_map, row), axis=1)
# Gschool = first_15_division(class_map, Gschool)

In [ ]:
# פונקציה לחלוקת התלמידים הנותרים שווה בשווה
Gschool = Gschool.apply(lambda row: equal_distribution_students(class_map, row), axis=1)
# Gschool = Gschool.apply(equal_distribution_students(class_map, Gschool), axis=1)

In [18]:
# הגדרת רשימת הכיתות לטווח גן עד יד'
class_map = ['כיתה_גן', 'כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']

# בדיקה בכל כיתה אם מספר התלמידים קטן מ-15 ועדכון ל-0
for single_class in class_map:
    Gschool[single_class] = Gschool[single_class].apply(lambda x: x if pd.isna(x) or x >= 15 else 0)

In [19]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']
}

# חישוב מספר התלמידים לכל שכבת גיל
Gschool = split_to_layers(Gschool, ages)

# חלוקה למגזרים
def set_sector(row):
    if row['מגזר'] == 'ערבי':
        return 1
    elif row['מגזר'] == 'יהודי':
        if row['סוג_פיקוח'] == 'חרדי':
            return 2
        elif row['סוג_פיקוח'] in ['ממלכתי', 'ממלכתי דתי']:
            return 3
    return None

Gschool['sector'] = Gschool.apply(set_sector, axis=1)

# שינוי שם של העמודות
Gschool.rename(columns={'סמל_מוסד': 'ID'}, inplace=True)
Gschool.rename(columns={'coordinate_x': 'x'}, inplace=True)
Gschool.rename(columns={'coordinate_y': 'y'}, inplace=True)

Gschool = Gschool.drop(['כיתה_גן', 'יישוב', 'כתובת', 'סוג_פיקוח', 'מגזר', 'יחידת_דיווח', 'שכבה', 'כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד', 'סה"כ_תלמידים', 'מכיתה', 'עד_כיתה', 'שם_מוסד', 'סוג_חינוך'], axis=1)

### ייצוא


In [20]:
Gschool.to_excel(r'{}\Intermediates\Gschool_final.xlsx'.format(cwd), index=False)